In [54]:
import csv
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift

In [55]:
# Load data from the train data csv file
Feature = []
Label = []
with open('data/train/train.csv', 'r') as f:
    reader = csv.reader(f)
    train_data = list(reader)
    # remove the first row
    train_data = train_data[1:]
    for row in train_data:
        Feature.append(np.array([float(x) for x in row[1:-1]]))
        Label.append(row[-1])  
Features = np.array(Feature)
Labels = np.array(Label)

In [56]:
# Load data from the test data csv file
test_features = []
with open('data/test/test.csv', 'r') as f:
    reader = csv.reader(f)
    test_data = list(reader)
    # remove the first row
    test_data = test_data[1:]
    for row in test_data:
        test_features.append(np.array([float(x) for x in row[1:]]))
test_features = np.array(test_features)

In [57]:
pca = PCA(n_components=415)
pca.fit(Features)
Features = pca.transform(Features)
test_features = pca.transform(test_features)
lda = LinearDiscriminantAnalysis()
lda.fit(Features, Labels)
Features = lda.transform(Features)
test_features = lda.transform(test_features)

In [58]:
# # implementing validation
# epsilon = 0.1 
# max_epsilon = 0
# max_min_pts = 0
# max_accuracy = 0
# while (epsilon<5):
#     for min_pts in range(17,25):
#         X_train, X_test, y_train, y_test = train_test_split(Features, Labels, test_size=0.2, random_state=42, stratify=Labels)
#         pca = PCA(n_components=415)
#         pca.fit(X_train)
#         X_train = pca.transform(X_train)
#         X_test = pca.transform(X_test)
#         lda = LinearDiscriminantAnalysis()
#         lda.fit(X_train, y_train)
#         X_train = lda.transform(X_train)
#         X_test = lda.transform(X_test)
#         clustering = DBSCAN(eps=epsilon, min_samples=min_pts)
#         cluster_labels = clustering.fit_predict(X_train)
#         cluster_labels = np.expand_dims(cluster_labels, axis=1)
#         X_train = np.concatenate((X_train, cluster_labels), axis=1)
#         cluster_labels = clustering.fit_predict(X_test)
#         cluster_labels = np.expand_dims(cluster_labels, axis=1)
#         X_test = np.concatenate((X_test, cluster_labels), axis=1)
#         logreg = LogisticRegression(max_iter=10000)
#         logreg.fit(X_train, y_train)
#         y_pred = logreg.predict(X_test)
#         print("epsilon: ", epsilon, "min_pts: ", min_pts, "accuracy: ", accuracy_score(y_test, y_pred))
#         if accuracy_score(y_test, y_pred) > max_accuracy:
#             max_accuracy = accuracy_score(y_test, y_pred)
#             max_epsilon = epsilon
#             max_min_pts = min_pts
#     epsilon += 0.1


In [59]:
# print("max_epsilon: ", max_epsilon, "max_min_pts: ", max_min_pts, "max_accuracy: ", max_accuracy)

In [60]:
# implementing dbscan clustering
clustering = DBSCAN(eps=0.5, min_samples=5)
cluster_labels = clustering.fit_predict(Features)
cluster_features = np.expand_dims(cluster_labels, axis=1)
Features = np.concatenate((Features, cluster_features), axis=1)


In [63]:
# implementing logistic regression
logreg = LogisticRegression(max_iter=10000)
logreg.fit(Features, Labels)
cluster_labels = clustering.fit_predict(test_features)
cluster_features = np.expand_dims(cluster_labels, axis=1)
test_features = np.concatenate((test_features, cluster_features), axis=1)
y_pred = logreg.predict(test_features)


In [ ]:
with open('data/test/submission_65.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Id', 'Category'])
    for i in range(len(y_pred)):
        writer.writerow([i, y_pred[i]])